In [2]:
import Pkg
Pkg.activate("/Users/alessiogiorlandino/Desktop/SAM/Project.toml")
Pkg.resolve()

  Activating project at `~/Desktop/SAM`


  No Changes to `~/Desktop/SAM/Project.toml`
  No Changes to `~/Desktop/SAM/Manifest.toml`


In [4]:
import SamApp
import Infernal, Rfam, FASTX
using BioSequences: LongRNA

    CondaPkg Found dependencies: /Users/alessiogiorlandino/.julia/packages/Logomaker/DdglG/CondaPkg.toml


    CondaPkg Found dependencies: /Users/alessiogiorlandino/.julia/packages/PythonCall/dsECZ/CondaPkg.toml


    CondaPkg Dependencies already up to date


In [ ]:
# CM model from Rfam (this has the noisy floor!)
Rfam_cm = Infernal.cmfetch(Rfam.cm(), "RF00162");

In [ ]:
# emit sequences from Rfam CM model
Rfam_cm_emitted_sequences_afa = Infernal.cmemit(Infernal.cmfetch(Rfam.cm(), "RF00162").out; N=10000, aligned=true, outformat="AFA");
Rfam_cm_emitted_sequences = FASTX.sequence.(FASTX.FASTA.Reader(open(Rfam_cm_emitted_sequences_afa.out)));
# remove inserts
Rfam_cm_emitted_sequences = [filter(!=('.'), filter(!islowercase, seq)) for seq in Rfam_cm_emitted_sequences];
@assert only(unique(length.(Rfam_cm_emitted_sequences))) == 108
Rfam_cm_emitted_sequences = LongRNA{4}.(Rfam_cm_emitted_sequences);

In [ ]:
# aligned hits, used to train a Refined CM model (in Stockholm format, without inserts!)
RF00162_hits_stk = Infernal.cmalign(Infernal.cmfetch(Rfam.cm(), "RF00162").out, Rfam.fasta_file("RF00162"); matchonly=true);
# fit refined CM model using full alignment (without inserts), and without entropic noise
Refined_cm = Infernal.cmbuild(RF00162_hits_stk.out; enone=true);

# emit sequences from Refined CM model
Refined_cm_emitted_sequences_afa = Infernal.cmemit(Refined_cm.cmout; N=10000, aligned=true, outformat="AFA");
Refined_cm_emitted_sequences = FASTX.sequence.(FASTX.FASTA.Reader(open(Refined_cm_emitted_sequences_afa.out)));
# remove inserts
Refined_cm_emitted_sequences = [filter(!=('.'), filter(!islowercase, seq)) for seq in Refined_cm_emitted_sequences];
@assert only(unique(length.(Refined_cm_emitted_sequences))) == 108
Refined_cm_emitted_sequences = LongRNA{4}.(Refined_cm_emitted_sequences);

In [ ]:
infernal_seq =SamApp.onehot(Refined_cm_emitted_sequences)